# Code pour plusieurs jours

In [ ]:
#pip install selenium
#pip install webdriver_manager
#pip install bs4

In [1]:
import datetime
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [10]:
# Créer une liste pour stocker les résultats de chaque jour
results = []

In [11]:
# Définir la date de début et la date de fin
start_date = datetime.date(2023, 1, 1)
end_date = datetime.date(2023, 4, 22)

In [12]:
# Boucle sur chaque jour
for i in range((end_date - start_date).days + 1):
    date = start_date + datetime.timedelta(days=i)
    since_date = date.strftime("%Y-%m-%d")
    until_date = (date + datetime.timedelta(days=1)).strftime("%Y-%m-%d")

    # Construire l'URL de la requête avec les dates du jour en cours
    url = f"https://nitter.net/search?f=tweets&q=Transition+%C3%A9cologique+ecologique&since={since_date}&until={until_date}&near="

    # Effectuer la requête HTTP et récupérer le contenu de la page
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # extraire les informations de chaque tweet pour le jour en cours
    tweet_contents = []
    tweet_dates = []
    tweet_links = []
    tweet_names = []
    comment_stats = []
    retweet_stats = []
    quote_stats = []
    heart_stats = []

    for tweet in soup.find_all('div', {'class': 'timeline-item'}):
        # contenu du tweet
        content = tweet.find('div', {'class': 'tweet-content'}).text.strip()
        tweet_contents.append(content)

        # date du tweet
        date = tweet.find('span', {'class': 'tweet-date'}).a['title']
        tweet_dates.append(date)

        # lien du tweet
        link = tweet.find('a', {'class': 'tweet-link'})['href']
        tweet_links.append(link)

        # nom (title) du tweet
        name = tweet.find('a', {'class': 'fullname'})['title']
        tweet_names.append(name)

        # statistiques
        stats = tweet.find_all('div', {'class': 'icon-container'})
        comment_stats.append(stats[0].text.strip())
        retweet_stats.append(stats[1].text.strip())
        quote_stats.append(stats[2].text.strip())
        heart_stats.append(stats[3].text.strip())

    # ajouter les informations du jour en cours à la liste de résultats
    result = pd.DataFrame({
        'Content': tweet_contents,
        'Date': tweet_dates,
        'Link': tweet_links,
        'Utilisateur': tweet_names,
        'Comments': comment_stats,
        'Retweets': retweet_stats,
        'Quotes': quote_stats,
        'Hearts': heart_stats
    })
    results.append(result)

In [13]:
# Concaténer les résultats de chaque jour dans un unique DataFrame
df = pd.concat(results)

In [14]:
# Fonction pour convertir la date dans le bon format
def convert_date(date_string):
    date_obj = datetime.datetime.strptime(date_string, "%b %d, %Y")
    return date_obj.strftime("%d/%m/%Y")

# Convertir la colonne 'Date' en format datetime
df['Date'] = pd.to_datetime(df['Date'], format='%b %d, %Y · %I:%M %p %Z')

# Créer les colonnes 'Jour', 'Mois' et 'Année'
df['Jour'] = df['Date'].dt.day
df['Mois'] = df['Date'].dt.month
df['Année'] = df['Date'].dt.year

# Convertir la colonne 'Date' en format JJ/MM/AAAA
df['Date'] = df['Date'].apply(lambda x: convert_date(x.strftime("%b %d, %Y")))

In [15]:
df = df.reindex(columns=['Content', 'Date', 'Jour', 'Mois', 'Année', 'Link', 'Utilisateur','Comments', 'Retweets', 'Quotes', 'Hearts'])

In [16]:
df

,Content,Date,Jour,Mois,Année,Link,Utilisateur,Comments,Retweets,Quotes,Hearts
0,"Ah, en effet...\nLa transition écologique est ...",01/01/2023,1,1,2023,/PaysagesIroise/status/1609699207169941505#m,Sauvegarde Paysages d'Iroise,,,,
1,Méthode pour une transition écologique dans le...,01/01/2023,1,1,2023,/InfosEducation/status/1609693388776906752#m,Education & Formation,,,,
2,Enseignement agricole et transition écologique...,01/01/2023,1,1,2023,/VIENSflashmober/status/1609686785084493826#m,Etat d'urgence,,,,
3,la pollueuse de notre planète qui a des intérê...,01/01/2023,1,1,2023,/LIBERTE1311/status/1609677427223007232#m,eddy,,,,
4,Quand la transition écologique bouscule la pol...,01/01/2023,1,1,2023,/PACOLLOT/status/1609676117555781634#m,Pierre-Alain Collot,,,,
...,...,...,...,...,...,...,...,...,...,...,...
13,ON NE JOUE PAS AVEC LA VIE DES GENS =\n--> On ...,22/04/2023,22,4,2023,/PandaGraou/status/1649826246828736514#m,PandaGraou 🐼🚂🐢,,,,
14,"Christophe Béchu : ""Je suis fier d'annoncer qu...",22/04/2023,22,4,2023,/VilainCat/status/1649825624020733960#m,Ministère des Bras Croisés 👻 ᵖᵃʳᵒᵈᶦᵉ,,5,131,2
15,100 propositions pour accélérer la transition ...,22/04/2023,22,4,2023,/CaboZpac/status/1649823147275821057#m,Bɑboum,,,,
16,"En fait, il faut imposer aux banques de financ...",22/04/2023,22,4,2023,/sterdyniak/status/1649816475023073284#m,Sterdyniak Henri,,1,,


In [17]:
import csv

df.to_csv(
    'PPD_Transition_Ecologique_Scrapping.csv',
    sep=';',
    index=False,
    encoding='utf-8',
    #quoting=csv.QUOTE_NONE,
    #escapechar='\\'
)

# Code avec 1 jour

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://nitter.net/search?f=tweets&q=transition+%C3%A9nerg%C3%A9tique+energetique&since=2023-04-20&until=2023-04-21&near=")
html = driver.page_source

In [ ]:
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
# extraire les informations de chaque tweet
tweet_contents = []
tweet_dates = []
tweet_links = []
tweet_names = []
comment_stats = []
retweet_stats = []
quote_stats = []
heart_stats = []

for tweet in soup.find_all('div', {'class': 'tweet-item'}):
    # contenu du tweet
    content = tweet.find('div', {'class': 'tweet-content'}).text.strip()
    tweet_contents.append(content)
    
    # date du tweet
    date = tweet.find('span', {'class': 'tweet-date'}).a['title']
    tweet_dates.append(date)
    
    # lien du tweet
    link = tweet.find('a', {'class': 'tweet-link'})['href']
    tweet_links.append(link)
    
    # nom (title) du tweet
    name = tweet.find('a', {'class': 'fullname'})['title']
    tweet_names.append(name)
    
    # statistiques
    stats = tweet.find_all('div', {'class': 'icon-container'})
    comment_stats.append(stats[0].text.strip())
    retweet_stats.append(stats[1].text.strip())
    quote_stats.append(stats[2].text.strip())
    heart_stats.append(stats[3].text.strip())

In [ ]:
# créer un dataframe pandas à partir des listes d'informations
df = pd.DataFrame({
    'Content': tweet_contents,
    'Date': tweet_dates,
    'Link': tweet_links,
    'Utilisateur': tweet_names,
    'Comments': comment_stats,
    'Retweets': retweet_stats,
    'Quotes': quote_stats,
    'Hearts': heart_stats
})

In [ ]:
# Fonction pour convertir la date dans le bon format
def convert_date(date_string):
    date_obj = datetime.datetime.strptime(date_string, "%b %d, %Y")
    return date_obj.strftime("%d/%m/%Y")

# Convertir la colonne 'Date' en format datetime
df['Date'] = pd.to_datetime(df['Date'], format='%b %d, %Y · %I:%M %p %Z')

# Créer les colonnes 'Jour', 'Mois' et 'Année'
df['Jour'] = df['Date'].dt.day
df['Mois'] = df['Date'].dt.month
df['Année'] = df['Date'].dt.year

# Convertir la colonne 'Date' en format JJ/MM/AAAA
df['Date'] = df['Date'].apply(lambda x: convert_date(x.strftime("%b %d, %Y")))

In [ ]:
df = df.reindex(columns=['Content', 'Date', 'Jour', 'Mois', 'Année', 'Link', 'Utilisateur','Comments', 'Retweets', 'Quotes', 'Hearts'])

In [ ]:
df

# Code pour plusieurs jours avec WebDriver

In [5]:
pip install webdriver_manager

In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup
import pandas as pd
import datetime

In [12]:
# Date de début de la recherche
start_date = datetime.date(2023, 4, 20)

# Date de fin de la recherche
end_date = datetime.date(2023, 4, 20)

# Créer la liste des dates à parcourir
delta = datetime.timedelta(days=1)
dates = []
while start_date <= end_date:
    dates.append(start_date)
    start_date += delta

In [23]:
# Initialiser le webdriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Liste des données à récupérer
tweet_contents = []
tweet_dates = []
tweet_links = []
tweet_names = []
comment_stats = []
retweet_stats = []
quote_stats = []
heart_stats = []

In [24]:
# Boucle pour parcourir les pages pour chaque date
for date in dates:
    url = f"https://nitter.net/search?f=tweets&q=transition+%C3%A9nerg%C3%A9tique+energetique&since={date}&until={date+delta}&near="
    driver.get(url)
    while True:
        # Extraire les informations de chaque tweet sur la page
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        for tweet in soup.find_all('div', {'class': 'timeline-item'}):
            # Contenu du tweet
            content = tweet.find('div', {'class': 'tweet-content'}).text.strip()
            tweet_contents.append(content)
            
            # Date du tweet
            date = tweet.find('span', {'class': 'tweet-date'}).a['title']
            tweet_dates.append(date)
            
            # Lien du tweet
            link = tweet.find('a', {'class': 'tweet-link'})['href']
            tweet_links.append(link)
            
            # Nom (title) du tweet
            name = tweet.find('a', {'class': 'fullname'})['title']
            tweet_names.append(name)
            
            # Statistiques
            stats = tweet.find_all('div', {'class': 'icon-container'})
            comment_stats.append(stats[0].text.strip())
            retweet_stats.append(stats[1].text.strip())
            quote_stats.append(stats[2].text.strip())
            heart_stats.append(stats[3].text.strip())
        
        # Cliquer sur le bouton "Load more" si présent
        try:
            #load_more_button = driver.find_element_by_class_name("show-more").find_element_by_tag_name("a")
            #load_more_button = driver.find_elements(By.CLASS_NAME, "show-more")
            #load_more_button=load_more_button.find_elements(By.TAG_NAME, "a")
            #print(load_more_button)
            #load_more_button.click()
            #print(load_more_button[0].text)
            #driver.execute_script("arguments[0].click();", load_more_button);
            
            # get element 
            element = driver.find_element_by_link_text("Load more")
            print(element)
            # click the element
            element.click()
            
        except:
            break

In [10]:
# Créer un dataframe pandas avec les données récupérées
df = pd.DataFrame({
    'Content': tweet_contents,
    'Date': tweet_dates,
    'Link': tweet_links,
    'Utilisateur': tweet_names,
    'Comments': comment_stats,
    'Retweets': retweet_stats,
    'Quotes': quote_stats,
    'Hearts': heart_stats
})

In [11]:
# Fonction pour convertir la date dans le bon format
def convert_date(date_string):
    date_obj = datetime.datetime.strptime(date_string, "%b %d, %Y")
    return date_obj.strftime("%d/%m/%Y")

# Convertir la colonne 'Date' en format datetime
df['Date'] = pd.to_datetime(df['Date'], format='%b %d, %Y · %I:%M %p %Z')

# Créer les colonnes 'Jour', 'Mois' et 'Année'
df['Jour'] = df['Date'].dt.day
df['Mois'] = df['Date'].dt.month
df['Année'] = df['Date'].dt.year

# Convertir la colonne 'Date' en format JJ/MM/AAAA
df['Date'] = df['Date'].apply(lambda x: convert_date(x.strftime("%b %d, %Y")))

In [12]:
df = df.reindex(columns=['Content', 'Date', 'Jour', 'Mois', 'Année', 'Link', 'Utilisateur','Comments', 'Retweets', 'Quotes', 'Hearts'])

In [13]:
df

,Content,Date,Jour,Mois,Année,Link,Utilisateur,Comments,Retweets,Quotes,Hearts
0,Transition énergétique = ARNAQUE !,10/04/2023,10,4,2023,/MarijkeF40/status/1645575957279801344#m,🇫🇷 Froidthier,,,,
1,"Je vais pas trop bien ces temps-ci, je fume co...",10/04/2023,10,4,2023,/DeInternaute/status/1645552789274218502#m,L’internaute de trop,,,,
2,#TransitionEnergetique Les défis qui attendent...,10/04/2023,10,4,2023,/SmartCityFrance/status/1645551811556679682#m,Smart City en France,,,,
3,#TransitionEnergetique Les défis qui attendent...,10/04/2023,10,4,2023,/SmartCityFrance/status/1645551768099602435#m,Smart City en France,,,,
4,Quand le monde raisonne a une transition énerg...,10/04/2023,10,4,2023,/ATshikumbi/status/1645550683456667650#m,evans kabuya,,,,
...,...,...,...,...,...,...,...,...,...,...,...
209,"Après le rejet de la loi sur le CO2, l'UDC veu...",20/04/2023,20,4,2023,/WeerachaiJaila1/status/1649109414048038918#m,Weerachai Jailangka,,,,
210,Juste avant de quitter @ZenonResearch j’ai été...,20/04/2023,20,4,2023,/Gregdt1/status/1649108074626416640#m,Greg De Temmerman,,,3,1
211,Concours Ministère de la Transition Energétiqu...,20/04/2023,20,4,2023,/tahiriyouness/status/1649108039700557824#m,أخبار الوظيفة,,,,
212,"Lu sur France 24 : ""TRANSITION ÉNERGÉTIQUE\nAr...",20/04/2023,20,4,2023,/31mai2022/status/1649107357220192256#m,Français Debout,,,,
